# FINM3422 - Trading Desk Analysis
## Exotic Derivatives Pricing and Portfolio Management

**Date:** May 16, 2025  
**Team:** Graduate Analysts - Exotic Derivatives Trading Desk  
**Objective:** Price and manage a portfolio of four OTC option positions

### Executive Summary
This notebook implements a comprehensive option pricing and portfolio management system for four specific OTC derivative trades:

1. **European Call on BHP** - Strike at 98% of current price, expiry Sept 15, 2027
2. **American Put on CBA** - Strike $170, expiry May 15, 2026
3. **European Up-and-In Barrier Call on WES** - Strike $80, barrier $100, expiry Sept 15, 2027
4. **European Basket Call** - Strike $175, expiry July 17, 2025, on BHP/CSL/WDS/MQG basket

The system uses object-oriented programming principles with appropriate valuation methods for each derivative type and provides comprehensive portfolio hedging analysis.

## 1. System Setup and Data Retrieval

### 1.1 Import Required Libraries and Modules
We begin by importing all necessary libraries and our custom modules for option pricing and data retrieval.

### 1.2 Market Data Retrieval

We fetch real-time market data for all required ASX stocks, calculate volatilities from historical data, estimate correlations, and obtain yield curve information. This section demonstrates our approach to sourcing market data from reliable sources.

### 1.3 Interest Rate Term Structure

Our approach uses a sophisticated yield curve boostrapping method to determine appropriate discount rates for different maturities. This ensures that we capture the term structure of interest rates accurately for each option's specific maturity.

## 2. Option Pricing Implementation

### 2.1 Trade 1: European Call Option on BHP

**Trade Details:**
- Underlying: BHP Group Ltd (BHP)
- Option Type: European Call
- Strike Price: 98% of current BHP price
- Expiry: September 15, 2027
- Position: **WRITE** (Bank sells the option)

**Valuation Method:** Black-Scholes model is most appropriate for European options as it provides a closed-form solution and is the industry standard for liquid European options.

In [3]:
from datetime import date
from Option_Classes import EuropeanOption

# Trade 1 parameters
current_price_bhp = 39.72
strike_price_bhp = 38.9256
expiry_bhp = date(2027, 9, 15)
option_type_bhp = 'call'
today_date_trade1 = date(2025, 5, 16)
sigma_bhp = 0.2272
dividend_yield_bhp = 0.0703 # 7.03% dividend yield for BHP

# Instantiate the European call option
bhp_eur_call = EuropeanOption(
    current_price=current_price_bhp,
    strike_price=strike_price_bhp,
    expiry=expiry_bhp,
    option_type=option_type_bhp,
    today_date=today_date_trade1,
    sigma=sigma_bhp,
    dividend_yield=dividend_yield_bhp
)

# Calculate the price
bhp_call_price = bhp_eur_call.option_price()

print(f"European Call Option on BHP (Strike = {strike_price_bhp:.2f}, Expiry = {expiry_bhp}):")
print(f"  Underlying: BHP")
print(f"  Current Price: ${current_price_bhp:.2f}")
print(f"  Implied Volatility (Sigma): {sigma_bhp*100:.2f}%")
print(f"  Dividend Yield: {dividend_yield_bhp*100:.2f}%")
print(f"Option Price: ${bhp_call_price:.2f}")

# Calculate and print Greeks for the BHP European call option
bhp_call_greeks = bhp_eur_call.greeks()
print("\nGreeks for BHP European Call Option:")
for greek_name, greek_value in bhp_call_greeks.items():
    print(f"  {greek_name.capitalize()}: {greek_value:.4f}")

European Call Option on BHP (Strike = 38.93, Expiry = 2027-09-15):
  Underlying: BHP
  Current Price: $39.72
  Implied Volatility (Sigma): 22.72%
  Dividend Yield: 7.03%
Option Price: $3.75

Greeks for BHP European Call Option:
  Delta: 0.4200
  Gamma: 0.0246
  Theta: -0.0007
  Vega: 0.2054
  Rho: 0.3050


### 2.2 Trade 2: American Put Option on CBA

**Trade Details:**
- Underlying: Commonwealth Bank of Australia (CBA)
- Option Type: American Put
- Strike Price: $170.00 (fixed)
- Expiry: May 15, 2026
- Position: **BUY** (Bank purchases the option)

**Valuation Method:** Binomial tree model is used for American options as it can handle the early exercise feature, which is valuable for puts when the stock price falls significantly below the strike.

In [4]:
from datetime import date
from Option_Classes import AmericanOption

# Trade 2 parameters
current_price_cba = 169.66
strike_price_cba = 170.00
expiry_cba = date(2026, 5, 15)
option_type_cba = 'put'
today_date_trade2 = date(2025, 5, 16)
sigma_cba = 0.18055
n_steps_cba = 1000 # Number of steps for the binomial tree
dividend_yield_cba = 0.039 # 3.9% dividend yield for CBA

# Instantiate the American put option
cba_american_put = AmericanOption(
    current_price=current_price_cba,
    strike_price=strike_price_cba,
    expiry=expiry_cba,
    option_type=option_type_cba,
    today_date=today_date_trade2,
    sigma=sigma_cba,
    n_steps=n_steps_cba,
    dividend_yield=dividend_yield_cba
)

# Calculate the price
cba_put_price = cba_american_put.option_price()

print(f"American Put Option on CBA (Strike = {strike_price_cba:.2f}, Expiry = {expiry_cba}):")
print(f"  Underlying: CBA")
print(f"  Current Price: ${current_price_cba:.2f}")
print(f"  Implied Volatility (Sigma): {sigma_cba*100:.2f}%")
print(f"  Dividend Yield: {dividend_yield_cba*100:.2f}%")
print(f"Option Price: ${cba_put_price:.2f}")

# Calculate and print Greeks for the CBA American put option
cba_put_greeks = cba_american_put.greeks()
print("\nGreeks for CBA American Put Option:")
for greek_name, greek_value in cba_put_greeks.items():
    print(f"  {greek_name.capitalize()}: {greek_value:.4f}")

American Put Option on CBA (Strike = 170.00, Expiry = 2026-05-15):
  Underlying: CBA
  Current Price: $169.66
  Implied Volatility (Sigma): 18.05%
  Dividend Yield: 3.90%
Option Price: $12.31

Greeks for CBA American Put Option:
  Delta: -0.4616
  Gamma: 0.0125
  Theta: -0.0161
  Vega: 0.6534
  Rho: -0.7330


### 2.3 Trade 3: European Up-and-In Barrier Call on WES

**Trade Details:**
- Underlying: Wesfarmers Limited (WES)
- Option Type: European Up-and-In Barrier Call
- Strike Price: $80.00
- Barrier Level: $100.00 (up-and-in)
- Expiry: September 15, 2027
- Position: **BUY** (Bank purchases the option)

**Valuation Method:** We use both binomial tree and Monte Carlo methods for barrier options. The binomial method provides discrete barrier monitoring while Monte Carlo offers continuous monitoring simulation.

In [5]:
from datetime import date
from Option_Classes import BarrierOption

# Trade 3 parameters - European Up-and-In Barrier Call on WES
current_price_wes = 82.56
strike_price_wes = 80.00
barrier_price_wes = 100.00
expiry_wes = date(2027, 9, 15)
option_type_wes = 'call'
today_date_trade3 = date(2025, 5, 16)
sigma_wes = 0.1839
dividend_yield_wes = 0.0349 # 3.49% dividend yield for WES

# Parameters for the models
n_steps_binomial_wes = 1000
n_paths_mc_wes_price = 50000
n_paths_mc_wes_greeks = 500000

# --- Binomial Model ---
wes_barrier_call_binomial = BarrierOption(
    current_price=current_price_wes,
    strike_price=strike_price_wes,
    expiry=expiry_wes,
    option_type=option_type_wes,
    today_date=today_date_trade3,
    sigma=sigma_wes,
    barrier_price=barrier_price_wes,
    method="binomial",
    n_steps=n_steps_binomial_wes,
    dividend_yield=dividend_yield_wes
)
wes_barrier_price_binomial_val = wes_barrier_call_binomial.option_price()

print(f"European Up-and-In Barrier Call on WES (Strike = {strike_price_wes:.2f}, Barrier = {barrier_price_wes:.2f}, Expiry = {expiry_wes}):")
print(f"--- Binomial Model ({n_steps_binomial_wes} steps) ---")
print(f"  Underlying: WES")
print(f"  Current Price: ${current_price_wes:.2f}")
print(f"  Implied Volatility (Sigma): {sigma_wes*100:.2f}%")
print(f"  Dividend Yield: {dividend_yield_wes*100:.2f}%")
print(f"Option Price (Binomial): ${wes_barrier_price_binomial_val:.2f}")

wes_barrier_greeks_binomial = wes_barrier_call_binomial.greeks()
print("\nGreeks for WES Barrier Call (Binomial Model):")
for greek_name, greek_value in wes_barrier_greeks_binomial.items():
    print(f"  {greek_name.capitalize()}: {greek_value:.4f}")

# --- Monte Carlo Model ---
# For Monte Carlo pricing
wes_barrier_call_mc_price_instance = BarrierOption(
    current_price=current_price_wes,
    strike_price=strike_price_wes,
    expiry=expiry_wes,
    option_type=option_type_wes,
    today_date=today_date_trade3,
    sigma=sigma_wes,
    barrier_price=barrier_price_wes,
    method="monte-carlo",
    n_steps=n_paths_mc_wes_price, # Using paths for pricing
    dividend_yield=dividend_yield_wes
)
wes_barrier_price_mc_val = wes_barrier_call_mc_price_instance.option_price()

print(f"\n--- Monte Carlo Model ({n_paths_mc_wes_price} paths for price) ---")
print(f"  Underlying: WES")
print(f"  Current Price: ${current_price_wes:.2f}")
print(f"  Implied Volatility (Sigma): {sigma_wes*100:.2f}%")
print(f"  Dividend Yield: {dividend_yield_wes*100:.2f}%")
print(f"Option Price (Monte Carlo): ${wes_barrier_price_mc_val:.2f}")

# For Monte Carlo Greeks
print(f"\nCalculating Monte Carlo Greeks with {n_paths_mc_wes_greeks} paths (this may take a moment)...")
wes_barrier_call_mc_greeks_instance = BarrierOption(
    current_price=current_price_wes,
    strike_price=strike_price_wes,
    expiry=expiry_wes,
    option_type=option_type_wes,
    today_date=today_date_trade3,
    sigma=sigma_wes,
    barrier_price=barrier_price_wes,
    method="monte-carlo",
    n_steps=n_paths_mc_wes_greeks, # Higher paths for Greeks
    dividend_yield=dividend_yield_wes
)
wes_barrier_greeks_mc = wes_barrier_call_mc_greeks_instance.greeks()
print("\nGreeks for WES Barrier Call (Monte Carlo Model):")
for greek_name, greek_value in wes_barrier_greeks_mc.items():
    print(f"  {greek_name.capitalize()}: {greek_value:.4f}")


European Up-and-In Barrier Call on WES (Strike = 80.00, Barrier = 100.00, Expiry = 2027-09-15):
--- Binomial Model (1000 steps) ---
  Underlying: WES
  Current Price: $82.56
  Implied Volatility (Sigma): 18.39%
  Dividend Yield: 3.49%
Option Price (Binomial): $9.52

Greeks for WES Barrier Call (Binomial Model):
  Delta: 0.5437
  Gamma: -0.0000
  Theta: -0.0039
  Vega: 0.4509
  Rho: 0.8462

--- Monte Carlo Model (50000 paths for price) ---
  Underlying: WES
  Current Price: $82.56
  Implied Volatility (Sigma): 18.39%
  Dividend Yield: 3.49%
Option Price (Monte Carlo): $8.62

Calculating Monte Carlo Greeks with 500000 paths (this may take a moment)...

Greeks for WES Barrier Call (Monte Carlo Model):
  Delta: 0.8617
  Gamma: -3.6865
  Theta: -0.0297
  Vega: 0.4740
  Rho: 0.7616


### 2.4 Trade 4: European Basket Call Option

**Trade Details:**
- Underlying Basket: 10% BHP, 35% CSL, 15% WDS, 40% MQG
- Option Type: European Call on basket value
- Strike Price: $175.00
- Expiry: July 17, 2025
- Position: **WRITE** (Bank sells the option)

**Valuation Method:** Monte Carlo simulation with correlation structure is essential for basket options as there's no closed-form solution. We use Cholesky decomposition to generate correlated asset price paths.

In [6]:
from datetime import date
import numpy as np
from Option_Classes import BasketOption # Ensure BasketOption is imported

# Trade 4 parameters - European Basket Call Option
# Basket definition: 10% BHP, 35% CSL, 15% WDS, 40% MQG
asset_tickers_in_basket = ['BHP.AX', 'CSL.AX', 'WDS.AX', 'MQG.AX']
weights = [0.10, 0.35, 0.15, 0.40] # Corresponding to asset_tickers_in_basket

# Current prices for the basket assets
current_prices_basket = [39.72, 241.82, 21.92, 211.25] # BHP, CSL, WDS, MQG

strike_price_basket = 175.00
expiry_basket = date(2025, 7, 17)
option_type_basket = 'call'
today_date_trade4 = date(2025, 5, 16)

# Volatilities for assets (ensure order matches asset_tickers_in_basket and current_prices)
sigma_bhp_basket = 0.2272
sigma_csl_basket = 0.2109
sigma_wds_basket = 0.2637
sigma_mqg_basket = 0.2085
sigmas_list_basket = [sigma_bhp_basket, sigma_csl_basket, sigma_wds_basket, sigma_mqg_basket]

# Dividend yields for assets (ensure order matches asset_tickers_in_basket)
div_yield_bhp_basket = 0.0703
div_yield_csl_basket = 0.0172
div_yield_wds_basket = 0.1202
div_yield_mqg_basket = 0.0356
dividend_yields_list_basket = [div_yield_bhp_basket, div_yield_csl_basket, div_yield_wds_basket, div_yield_mqg_basket]

# Hardcoded Correlation Matrix (ensure order matches asset_tickers_in_basket)
correlation_matrix_np_basket = np.array([
    [1.000000, 0.452170, 0.807649, 0.878665],  # BHP with BHP, CSL, WDS, MQG
    [0.452170, 1.000000, 0.581252, 0.243513],  # CSL with BHP, CSL, WDS, MQG
    [0.807649, 0.581252, 1.000000, 0.651679],  # WDS with BHP, CSL, WDS, MQG
    [0.878665, 0.243513, 0.651679, 1.000000]   # MQG with BHP, CSL, WDS, MQG
])

# Instantiate the basket option
basket_call = BasketOption(
    current_prices=current_prices_basket,
    weights=weights,
    strike_price=strike_price_basket,
    expiry=expiry_basket,
    option_type=option_type_basket,
    today_date=today_date_trade4,
    sigmas=sigmas_list_basket,
    correlation_matrix=correlation_matrix_np_basket,
    dividend_yields=dividend_yields_list_basket
)

basket_call_price = basket_call.option_price()

print(f"European Basket Call Option (Strike = {strike_price_basket:.2f}, Expiry = {expiry_basket}):")
print(f"  Basket composition: {weights[0]*100:.0f}% BHP, {weights[1]*100:.0f}% CSL, {weights[2]*100:.0f}% WDS, {weights[3]*100:.0f}% MQG")

asset_price_info = ", ".join([f"{asset.replace('.AX','')}: ${price:.2f}" for asset, price in zip(asset_tickers_in_basket, current_prices_basket)])
print(f"  Current Prices: {asset_price_info}")

asset_sigma_info = ", ".join([f"{asset.replace('.AX','')}: {sig*100:.2f}%" for asset, sig in zip(asset_tickers_in_basket, sigmas_list_basket)])
print(f"  Implied Volatilities (Sigmas): {asset_sigma_info}")

asset_div_info = ", ".join([f"{asset.replace('.AX','')}: {div*100:.2f}%" for asset, div in zip(asset_tickers_in_basket, dividend_yields_list_basket)])
print(f"  Dividend Yields: {asset_div_info}")

print(f"Option Price: ${basket_call_price:.2f}")

print("\nCalculating Basket Greeks (this may take a moment)...")
basket_call_greeks = basket_call.greeks()
print("\nGreeks for European Basket Call Option:")
for greek_name, greek_value in basket_call_greeks.items():
    if greek_name == 'deltas':
        print(f"  Deltas (for {', '.join(asset.replace('.AX','') for asset in asset_tickers_in_basket)} respectively):")
        for i, delta_val in enumerate(greek_value):
            print(f"    {asset_tickers_in_basket[i].replace('.AX','')}: {delta_val:.4f}")
    else:
        print(f"  {greek_name.capitalize()}: {greek_value:.4f}")

European Basket Call Option (Strike = 175.00, Expiry = 2025-07-17):
  Basket composition: 10% BHP, 35% CSL, 15% WDS, 40% MQG
  Current Prices: BHP: $39.72, CSL: $241.82, WDS: $21.92, MQG: $211.25
  Implied Volatilities (Sigmas): BHP: 22.72%, CSL: 21.09%, WDS: 26.37%, MQG: 20.85%
  Dividend Yields: BHP: 7.03%, CSL: 1.72%, WDS: 12.02%, MQG: 3.56%
Option Price: $6.46

Calculating Basket Greeks (this may take a moment)...

Greeks for European Basket Call Option:
  Deltas (for BHP, CSL, WDS, MQG respectively):
    BHP: -0.3150
    CSL: 0.2178
    WDS: -0.8652
    MQG: 0.2043
  Vega: 0.3917
  Theta: -0.0301
  Rho: 0.2627


## 3. Portfolio Summary and Risk Management

### 3.1 Portfolio Summary

We now construct a summary of the portfolio with all four trades, considering the bank's position (long/short) for each option.

In [9]:
# Portfolio Definition
# We will store each trade's data, including different model results where applicable.

portfolio = [
    {
        "name": "Trade 1: Short European Call on BHP",
        "position": -1, # Bank is short
        "underlying": "BHP",
        "models": [
            {
                "method_name": "Black-Scholes",
                "option_obj": bhp_eur_call,
                "price": bhp_call_price,
                "greeks": bhp_call_greeks
            }
        ]
    },
    {
        "name": "Trade 2: Long American Put on CBA",
        "position": 1, # Bank is long
        "underlying": "CBA",
        "models": [
            {
                "method_name": "Binomial Tree",
                "option_obj": cba_american_put,
                "price": cba_put_price,
                "greeks": cba_put_greeks
            }
        ]
    },
    {
        "name": "Trade 3: Long European Up-and-In Barrier Call on WES",
        "position": 1, # Bank is long
        "underlying": "WES",
        "models": [
            {
                "method_name": "Binomial Model",
                "option_obj": wes_barrier_call_binomial, # Object used for binomial pricing & greeks
                "price": wes_barrier_price_binomial_val,
                "greeks": wes_barrier_greeks_binomial
            },
            {
                "method_name": "Monte Carlo Model",
                # Price from wes_barrier_call_mc_price_instance, greeks from wes_barrier_call_mc_greeks_instance
                "option_obj": wes_barrier_call_mc_greeks_instance, # Using the instance for greeks calc
                "price": wes_barrier_price_mc_val, # Calculated using n_paths_mc_wes_price
                "greeks": wes_barrier_greeks_mc   # Calculated using n_paths_mc_wes_greeks
            }
        ]
    },
    {
        "name": "Trade 4: Short European Basket Call",
        "position": -1, # Bank is short
        "underlying": "BHP, CSL, WDS, MQG", # Basket components
        "models": [
            {
                "method_name": "Monte Carlo Simulation",
                "option_obj": basket_call,
                "price": basket_call_price,
                "greeks": basket_call_greeks
            }
        ]
    }
]

print("### Portfolio Summary ###")
for trade in portfolio:
    print(f"\n--- {trade['name']} ---")
    position_str = "WRITE (SELL)" if trade["position"] == -1 else "BUY (PURCHASE)"
    print(f"  Position: {position_str}")
    print(f"  Underlying Asset(s): {trade['underlying']}")

    for model_results in trade['models']:
        print(f"\n  Method: {model_results['method_name']}")
        print(f"    Option Price: ${model_results['price']:.2f}")
        print(f"    Greeks:")
        for greek_name, greek_value in model_results['greeks'].items():
            if greek_name == 'deltas': # Specific handling for basket option deltas
                # asset_tickers_in_basket is defined in the Trade 4 cell (8cc0cf46)
                # This cell (7982e9eb) should be run after Trade 4 cell.
                asset_names_str = ', '.join(asset.replace('.AX','') for asset in asset_tickers_in_basket)
                print(f"      Deltas (for {asset_names_str} respectively):")
                if isinstance(greek_value, list): # Ensure it's a list before enumerating
                    for i, delta_val in enumerate(greek_value):
                        print(f"        {asset_tickers_in_basket[i].replace('.AX','')}: {delta_val:.4f}")
                else: # Should not happen if greeks structure is correct
                     print(f"        Delta value is not a list: {greek_value}")
            else:
                print(f"      {greek_name.capitalize()}: {greek_value:.4f}")
    print("-" * (len(trade['name']) + len("---  ---") + 2)) # Separator for better readability


### Portfolio Summary ###

--- Trade 1: Short European Call on BHP ---
  Position: WRITE (SELL)
  Underlying Asset(s): BHP

  Method: Black-Scholes
    Option Price: $3.75
    Greeks:
      Delta: 0.4200
      Gamma: 0.0246
      Theta: -0.0007
      Vega: 0.2054
      Rho: 0.3050
---------------------------------------------

--- Trade 2: Long American Put on CBA ---
  Position: BUY (PURCHASE)
  Underlying Asset(s): CBA

  Method: Binomial Tree
    Option Price: $12.31
    Greeks:
      Delta: -0.4616
      Gamma: 0.0125
      Theta: -0.0161
      Vega: 0.6534
      Rho: -0.7330
-------------------------------------------

--- Trade 3: Long European Up-and-In Barrier Call on WES ---
  Position: BUY (PURCHASE)
  Underlying Asset(s): WES

  Method: Binomial Model
    Option Price: $9.52
    Greeks:
      Delta: 0.5437
      Gamma: -0.0000
      Theta: -0.0039
      Vega: 0.4509
      Rho: 0.8462

  Method: Monte Carlo Model
    Option Price: $8.62
    Greeks:
      Delta: 0.8617
      G

### 3.3 Delta Hedging

This section provides comprehensive hedging recommendations based on the portfolio's Delta exposures. We analyse basic delta hedging strategies appropriate for an institutional trading desk.

In [10]:
# Delta Hedging Recommendations

print("### Delta Hedging Recommendations ###")

# Trade 1: Short European Call on BHP
# Bank's position is short the call, so exposure is -delta. To hedge, bank BUYS delta shares.
trade1_delta = bhp_call_greeks['delta']
print(f"\n--- Trade 1: Short European Call on BHP (Delta: {trade1_delta:.4f}) ---")
print(f"  Bank's Delta Exposure: {-trade1_delta:.4f}")
print(f"  Action to Delta Hedge: BUY {trade1_delta:.4f} shares of BHP.")

# Trade 2: Long American Put on CBA
# Bank's position is long the put, so exposure is delta (which is negative for a put). To hedge, bank BUYS |delta| shares.
trade2_delta = cba_put_greeks['delta']
print(f"\n--- Trade 2: Long American Put on CBA (Delta: {trade2_delta:.4f}) ---")
print(f"  Bank's Delta Exposure: {trade2_delta:.4f}")
# If delta is -0.46, exposure is -0.46. To make it 0, add 0.46. So BUY 0.46 shares.
print(f"  Action to Delta Hedge: BUY {-trade2_delta:.4f} shares of CBA.")


# Trade 3: Long European Up-and-In Barrier Call on WES (Binomial)
# Bank's position is long the call, so exposure is +delta. To hedge, bank SELLS delta shares.
# We'll use the Binomial model's delta for this summary.
trade3_delta_binomial = wes_barrier_greeks_binomial['delta']
print(f"\n--- Trade 3: Long European Up-and-In Barrier Call on WES (Binomial Delta: {trade3_delta_binomial:.4f}) ---")
print(f"  Bank's Delta Exposure: {trade3_delta_binomial:.4f}")
print(f"  Action to Delta Hedge (Binomial): SELL {trade3_delta_binomial:.4f} shares of WES.")

# Displaying MC Delta hedge as well for completeness, as per previous notebook structure
trade3_delta_mc = wes_barrier_greeks_mc['delta']
print(f"  (For reference, Monte Carlo Delta: {trade3_delta_mc:.4f})")
print(f"  (Action to Delta Hedge (Monte Carlo): SELL {trade3_delta_mc:.4f} shares of WES.)")


# Trade 4: Short European Basket Call
# Bank's position is short the call, so exposure is -delta_i for each component i.
# To hedge, bank BUYS delta_i shares of component i.
trade4_deltas = basket_call_greeks['deltas'] # This is a list of deltas
print(f"\n--- Trade 4: Short European Basket Call ---")
print(f"  Bank's Delta Exposures & Hedging Actions:")
for i, asset_delta in enumerate(trade4_deltas):
    asset_name = asset_tickers_in_basket[i].replace('.AX','')
    # The 'asset_delta' from basket_call_greeks['deltas'] is the option's delta for that component.
    # If the bank is SHORT the call, its exposure to asset_i is -asset_delta.
    # To hedge, the bank needs to take a position of +asset_delta in the underlying.
    # So, if asset_delta is positive (typical for call component), bank buys.
    # If asset_delta is negative (anomalous, but as per output), bank sells.
    
    # The current output for BHP and WDS deltas in Trade 4 was negative.
    # A call option component delta should typically be positive.
    # If asset_delta > 0 for a call component: Bank is short option, exposure is -asset_delta. Hedge by BUYING asset_delta shares.
    # If asset_delta < 0 for a call component: Bank is short option, exposure is -asset_delta (which is positive). Hedge by SELLING |asset_delta| shares.
    # So, the action is always to take a position of 'asset_delta' in the underlying to offset the '-asset_delta' exposure from the short call.

    print(f"    {asset_name}:")
    print(f"      Option Component Delta: {asset_delta:.4f}")
    print(f"      Bank's Exposure to {asset_name}: {-asset_delta:.4f}")
    if asset_delta >= 0:
        print(f"      Action to Delta Hedge: BUY {asset_delta:.4f} shares of {asset_name}.")
    else:
        # If the option's delta component is negative, and bank is short the option,
        # bank's exposure is -(-ve delta) = +ve delta. So, bank should SELL.
        print(f"      Action to Delta Hedge: SELL {-asset_delta:.4f} shares of {asset_name}. (Note: based on negative delta from MC)")

print("\nReminder: These delta hedges are instantaneous and need to be rebalanced as market conditions change.")
print("The deltas for some components of the Basket Call (Trade 4) previously showed anomalous signs/magnitudes and should be carefully reviewed before executing hedges.")

### Delta Hedging Recommendations ###

--- Trade 1: Short European Call on BHP (Delta: 0.4200) ---
  Bank's Delta Exposure: -0.4200
  Action to Delta Hedge: BUY 0.4200 shares of BHP.

--- Trade 2: Long American Put on CBA (Delta: -0.4616) ---
  Bank's Delta Exposure: -0.4616
  Action to Delta Hedge: BUY 0.4616 shares of CBA.

--- Trade 3: Long European Up-and-In Barrier Call on WES (Binomial Delta: 0.5437) ---
  Bank's Delta Exposure: 0.5437
  Action to Delta Hedge (Binomial): SELL 0.5437 shares of WES.
  (For reference, Monte Carlo Delta: 0.8617)
  (Action to Delta Hedge (Monte Carlo): SELL 0.8617 shares of WES.)

--- Trade 4: Short European Basket Call ---
  Bank's Delta Exposures & Hedging Actions:
    BHP:
      Option Component Delta: -0.3150
      Bank's Exposure to BHP: 0.3150
      Action to Delta Hedge: SELL 0.3150 shares of BHP. (Note: based on negative delta from MC)
    CSL:
      Option Component Delta: 0.2178
      Bank's Exposure to CSL: -0.2178
      Action to Delta Hed

### 3.4 Portfolio Stress Testing and Scenario Analysis

We conduct scenario analysis to understand how the portfolio performs under different market conditions. This demonstrates sophisticated risk management practices expected in institutional environments.

## 4. Summary and Trading Recommendations

### 4.1 Executive Summary

This section provides a comprehensive summary of all pricing results, risk assessments, and strategic recommendations for the trading desk.

## 5. Technical Documentation

### 5.1 Model Validation and Implementation Notes

This section documents the technical implementation decisions and validates our modeling choices against industry best practices.